In [ ]:
using Revise
using Plots
# pyplot()
plotly()

using QSimulator

# Transmon Ramsey Oscillations

Starting from a coherence between two energy eigenstates we expect oscilltions at the detuning between the energy levels.  We'll work with natural units of GHz and ns for numerical stability reasons. We'll start with a single 3 level transmon in the lab frame with a 5 GHz qubit frequency and -200 MHz anharmonicity.

In [ ]:
# we create a specific QSystem
q0 = FixedDuffingTransmon("q0", 5, -0.2, 3)
# we can ask for the Hamiltonian of an QSystem as a Matrix
hamiltonian(q0)

In [ ]:
# as CompositeQSystems is a tensor product structure of QSystem's and is what all the solvers are built around
cqs = CompositeQSystem([q0]);
add_hamiltonian!(cqs, q0)
hamiltonian(cqs)

In [ ]:
# evolve an initial superposition state for 1 ns
times = collect(linspace(0,1,201))
ψ0 = (1/sqrt(2)) * Complex128[1; 1; 0]
ψs = unitary_state(cqs, times, ψ0);

In [ ]:
# plot the projection on to the initial state and we expect to see 5 GHz oscillations
signal = Float64[real(ψ0'*ψ) for ψ in ψs]
expected = 0.5 + 0.5*cos.(2π*5 * (linspace(0,1,201)))
p = plot(times, signal, linewidth=2, label="simulated")
plot!(p, times, expected, label="ideal")
xlabel!(p, "Time (ns)")

# Rabi Oscillations

Driving Rabi oscillations in the lab frame is a good example of a parametric time dependent Hamiltonian. The drive electric field couples to the transmon dipole or $X$ operator.

## Constant Drive

In [ ]:
qubit_freq = 5.0
q0 = FixedDuffingTransmon("q0", qubit_freq, -0.2, 3)
cqs = CompositeQSystem([q0]);
add_hamiltonian!(cqs, q0)
add_hamiltonian!(cqs, microwave_drive(q0, t -> 0.02*cos(2π*qubit_freq * t)), q0);
ψ_init = Complex128[1; 0; 0]
times = collect(linspace(0,100,101))
ψs = unitary_state(cqs, times, ψ_init);

In [ ]:
p = plot(times, [abs2(s[1]) for s in ψs], label="Ground State Pop.")
plot!(p, times, [abs2(s[2]) for s in ψs], label="Excited State Pop.")
xlabel!(p, "Time (ns)")

## Variable Amplitude Gaussian Pulse

In [ ]:
# write a helper function that returns the drive Hamiltonian at a particular point in time
function gaussian(pulse_length, pulse_freq, t; cutoff=2.5)
    σ = pulse_length/2/cutoff
    pulse = exp(-0.5*((t-pulse_length/2)/σ)^2)
    pulse * cos(2π*pulse_freq * t)
end

function flat(pulse_freq,  t)
   cos(2π*pulse_freq * t)
end

In [ ]:
ψ0 = Complex128[1; 0; 0]
states_flat = []
states_gaussian = []
amps = 0.1 * linspace(0,1, 51)
pulse_length = 25.0
qubit_freq = 5.0

for amp = amps
    # first do flat pulse
    # three level transmon in the lab frame
    q0 = FixedDuffingTransmon("q0", qubit_freq, -0.2, 3)
    cqs = CompositeQSystem([q0]);
    add_hamiltonian!(cqs, q0)
    add_hamiltonian!(cqs, microwave_drive(q0, t -> amp*flat(qubit_freq, t)), q0);
    ψs = unitary_state(cqs, [0, pulse_length], ψ0)
    push!(states_flat, ψs[end])

    # now gaussian
    cqs = CompositeQSystem([q0]);
    add_hamiltonian!(cqs, q0)
    add_hamiltonian!(cqs, microwave_drive(q0, t -> amp*gaussian(pulse_length, qubit_freq, t)), q0);
    ψs = unitary_state(cqs, [0, pulse_length], ψ0)
    push!(states_gaussian, ψs[end])
end

In [ ]:
p1 = plot()
for ct = 0:2
    plot!(p1, amps*1e3, [abs2(s[ct+1]) for s in states_flat], label="$ct State Pop.")
end
xlabel!(p1, "Nutation Strength (MHz)")
title!(p1, "Flat Pulse")
p2 = plot()
for ct = 0:2
    plot!(p2, amps*1e3, [abs2(s[ct+1]) for s in states_gaussian], label="$ct State Pop.")
end
xlabel!(p2, "Peak Nutation Strength (MHz)")
title!(p2, "Gaussian Pulse")
plot(p1,p2, layout=(1,2), size=(800,400))


# Two Qubit  Gates - Parametric Gates in the Lab Frame

$$ \mathcal{H}(t) = \omega_0 \hat{n}_0 + \Delta_0 \Pi_{2_0} + \omega_1(t)\hat{n}_1 + \Delta_1 \Pi_{2_1} + gX_0X_1$$

## iSWAP

In [ ]:
# parameters from Blue Launch paper
q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)

freqs = 117:0.5:127
times = collect(0.0:5:1000)
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[1.0; 0.0; 0.0] # start in 10 state
pop_01 = []
pop_10 = []

amp = 0.323

for freq = freqs
    cqs = CompositeQSystem([q0, q1])
    add_hamiltonian!(cqs, q0)
    add_hamiltonian!(cqs, 0.006*dipole(q0, q1), [q0,q1])
    add_hamiltonian!(cqs, flux_drive(q1, t -> amp*sin(2π*freq/1e3*t)), q1)
    ψs = unitary_state(cqs, times, ψ0);
    push!(pop_01, [abs2(ψ[2]) for ψ in ψs])
    push!(pop_10, [abs2(ψ[4]) for ψ in ψs])
end

In [ ]:
p1 = contour(freqs, times, cat(2, pop_01...), fill=true)
xlabel!(p1, "Frequency (MHz)")
ylabel!(p1, "Time (ns)")
p2 = contour(freqs, times, cat(2, pop_10...), fill=true)
xlabel!(p2, "Frequency (MHz)")
plot(p1,p2, layout=(1,2), size=(600,300))

# CZ02

In [ ]:
# parameters from Blue Launch paper
q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)

freqs = 110:0.5:120
times = collect(0.0:5:1000)
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[0.0; 1.0; 0.0] # start in 11 state
pop_11 = []
pop_02 = []

amp = 0.245

for freq = freqs
    cqs = CompositeQSystem([q0, q1])
    add_hamiltonian!(cqs, q0)
    add_hamiltonian!(cqs, 0.006*dipole(q0, q1), [q0,q1])
    add_hamiltonian!(cqs, flux_drive(q1, t -> amp*sin(2π*freq/1e3*t)), q1)
    ψs = unitary_state(cqs, times, ψ0);
    push!(pop_11, [abs2(ψ[5]) for ψ in ψs])
    push!(pop_02, [abs2(ψ[3]) for ψ in ψs])
end
pop_11 = cat(2, pop_11...)
pop_02 = cat(2, pop_02...);

In [ ]:
p1 = contour(freqs, times, pop_11, fill=true)
xlabel!(p1, "Frequency (MHz)")
ylabel!(p1, "Time (ns)")
p2 = contour(freqs, times, pop_02, fill=true)
xlabel!(p2, "Frequency (MHz)")
plot(p1,p2, layout=(1,2), size=(600,300))

In [ ]:
# Look more finely at a slice along time to show we are getting full contrast and look at lab frame jaggedness
q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)

# should get an iSWAP interaction at ≈ 122 MHz
freq = 122.1/1e3
amp = 0.323
times = collect(0:200)
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[1.0; 0.0; 0.0] # start in 10 state

cqs = CompositeQSystem([q0, q1])
add_hamiltonian!(cqs, q0)
add_hamiltonian!(cqs, 0.006*dipole(q0, q1), [q0,q1])
add_hamiltonian!(cqs, flux_drive(q1, t -> amp*sin(2π*freq*t)), q1)
ψs = unitary_state(cqs, times, ψ0);
pop_01 = [abs2(ψ[2]) for ψ in ψs]
pop_10 = [abs2(ψ[4]) for ψ in ψs];

In [ ]:
p = plot(times, pop_01, label="01")
plot!(p, times, pop_10, label="10")

# Two Qubit  Gates - Parametric Gates in the Rotating Frame

We can move into the doubly rotating frame. The dipole coupling becomes time dependent and we discard the flip-flip (flop-flop) terms in the Hamiltonian.

$$ \mathcal{H}(t) =  \Delta_0 \Pi_{2_0} + \omega_1(t)\hat{n}_1 + \Delta_1 \Pi_{2_1} - \omega_1(0)\hat{n}_1 + e^{i\delta t}\sigma_+\sigma_- + e^{-i\delta t}\sigma_-\sigma_+$$

In [ ]:
q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)

# should get an iSWAP interaction at ≈ 122 MHz
freq = 122.1/1e3
amp = 0.323
times = collect(0:200)
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[1.0; 0.0; 0.0] # start in 10 state

cqs = CompositeQSystem([q0, q1])
add_hamiltonian!(cqs, q0)
# add rotating frame Hamiltonian shifts
add_hamiltonian!(cqs, -q0.frequency*number(q0), q0)
q1_freq = hamiltonian(q1, 0.0)[2,2]
add_hamiltonian!(cqs, -q1_freq*number(q1), q1)

diff_freq = q0.frequency - q1_freq
add_hamiltonian!(cqs,
                (ham,idxs,t) -> QSimulator.embed_add!(ham, (2π*0.006)*flip_flop(q0,q1; ϕ=diff_freq*t), idxs),
                [q0,q1])
add_hamiltonian!(cqs, flux_drive(q1, t -> amp*sin(2π*freq*t)), q1)

ψs = unitary_state(cqs, times, ψ0);
pop_01 = [abs2(ψ[2]) for ψ in ψs]
pop_10 = [abs2(ψ[4]) for ψ in ψs];

In [ ]:
p = plot(times, pop_01, label="01")
plot!(p, times, pop_10, label="10")

In [ ]:
q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)

# should get an CZ02 interaction at ≈ 115 MHz
freq = 115.5/1e3
amp = 0.245
times = collect(0:0.5:200)
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[0.0; 1.0; 0.0] # start in 11 state

cqs = CompositeQSystem([q0, q1])
add_hamiltonian!(cqs, q0)
# add rotating frame Hamiltonian shifts
add_hamiltonian!(cqs, -q0.frequency*number(q0), q0)
q1_freq = hamiltonian(q1, 0.0)[2,2]
add_hamiltonian!(cqs, -q1_freq*number(q1), q1)

diff_freq = q0.frequency - q1_freq
add_hamiltonian!(cqs, rotating_flip_flop(q0, q1, 0.006, diff_freq), [q0,q1])
add_hamiltonian!(cqs, flux_drive(q1, t -> amp*sin(2π*freq*t)), q1)

ψs = unitary_state(cqs, times, ψ0);
pop_11 = [abs2(ψ[5]) for ψ in ψs]
pop_02 = [abs2(ψ[3]) for ψ in ψs];

In [ ]:
p = plot(times, pop_11, label="11")
plot!(p, times, pop_02, label="02")

# Two Qubit  Gates - Soft Shoulders

Look at how soft shoulders distort the pulse shape.


In [ ]:
q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)

# should get an CZ02 interaction at ≈ 115 MHz
freqs = 1e-3*(110:0.5:120)
amp = 0.245
risetime = 50
times = collect(2*risetime:5:500)
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[0.0; 1.0; 0.0] # start in 11 state

pop_11 = Float64[]
pop_02 = Float64[]

for tmax = times

    # erfsquared pulse parameters
    fwhm = 0.5 * risetime
    t₁ = fwhm
    t₂ = tmax - fwhm
    σ = 0.5 * fwhm / sqrt(2*log(2))
    erf_squared(t) = 0.5 * (erf((t - t₁)/σ) - erf((t - t₂)/σ) )

    for freq = freqs
        cqs = CompositeQSystem([q0, q1])
        add_hamiltonian!(cqs, q0)
        # add rotating frame Hamiltonian shifts
        add_hamiltonian!(cqs, -q0.frequency*number(q0), q0)
        q1_freq = hamiltonian(q1, 0.0)[2,2]
        add_hamiltonian!(cqs, -q1_freq*number(q1), q1)

        diff_freq = q0.frequency - q1_freq
        add_hamiltonian!(cqs, rotating_flip_flop(q0, q1, 0.006, diff_freq), [q0,q1])

        add_hamiltonian!(cqs, flux_drive(q1, t -> amp*erf_squared(t)*sin(2π*freq*t)), q1)

        ψ = unitary_state(cqs, float(tmax), ψ0);
        push!(pop_11, abs2(ψ[5]))
        push!(pop_02, abs2(ψ[3]));
    end
end
pop_11 = reshape(pop_11, length(freqs), length(times))
pop_02 = reshape(pop_02, length(freqs), length(times));

In [ ]:
p1 = contour(times, 1e3*freqs, pop_11, fill=true)
xlabel!(p1, "Frequency (MHz)")
ylabel!(p1, "Time (ns)")
title!(p1, "Population 11")
p2 = contour(times, 1e3*freqs, pop_02, fill=true)
xlabel!(p2, "Frequency (MHz)")
title!(p2,  "Population 02")
plot(p1,p2, layout=(1,2), size=(800,400))

# Pulse Amplitude Noise

We can estimate the effect of fluctuations on the pulse amplitude with a Krauss map sum of unitaries weighted by a normal distribution.

$$ \mathcal{S}(\rho) = \sum_k \lambda_k U_k\rho U_k^\dagger $$

In [ ]:
# pick an operating point of the plots above

freq = 115/1e3
amp = 0.245
tmax = 205

q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)

# should get an CZ02 interaction at ≈ 115 MHz
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[0.0; 1.0; 0.0] # start in 11 state

risetime = 50
fwhm = 0.5 * risetime
t₁ = fwhm
t₂ = tmax - fwhm
σ = 0.5 * fwhm / sqrt(2*log(2))
erf_squared(t) = 0.5 * (erf((t - t₁)/σ) - erf((t - t₂)/σ) )

cqs = CompositeQSystem([q0, q1])
add_hamiltonian!(cqs, q0)
# add rotating frame Hamiltonian shifts
add_hamiltonian!(cqs, -q0.frequency*number(q0), q0)
q1_freq = hamiltonian(q1, 0.0)[2,2]
add_hamiltonian!(cqs, -q1_freq*number(q1), q1)

diff_freq = q0.frequency - q1_freq
add_hamiltonian!(cqs, rotating_flip_flop(q0, q1, 0.006, diff_freq), [q0,q1])

add_hamiltonian!(cqs, flux_drive(q1, t -> amp*erf_squared(t)*sin(2π*freq*t)), q1)

U = unitary_propagator(cqs, float(tmax));

In [ ]:
# project out to qubit subspace 
function project_qubit_subspace(U)
    const basis_idx = [1,2,4,5]
    U_proj = zeros(Complex128, (4,4))
    for (ct1,idx1) = enumerate(basis_idx), (ct2,idx2) = enumerate(basis_idx)
        U_proj[ct1,ct2] = U[idx1,idx2]
    end
    U_proj
end
U_proj = project_qubit_subspace(U)

In [ ]:
import QuantumInfo: liou, avgfidelity, kraus2liou
import Cliffords: Z
using Optim

In [ ]:
Zrot = θ -> expm(-1im * θ * π * Z)

In [ ]:
CZ = diagm([1.0,1.0,1.0,-1.0])
res = optimize(zs -> 1 - avgfidelity(liou((Zrot(zs[1]) ⊗ Zrot(zs[2])) * U_proj), liou(CZ)), [0.0, 0.0])

In [ ]:
# pick an operating point of the plots above
freq = 115/1e3
amp = 0.245
tmax = 205

q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)

# should get an CZ02 interaction at ≈ 115 MHz
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[0.0; 1.0; 0.0] # start in 11 state

risetime = 50
fwhm = 0.5 * risetime
t₁ = fwhm
t₂ = tmax - fwhm
σ = 0.5 * fwhm / sqrt(2*log(2))
erf_squared(t) = 0.5 * (erf((t - t₁)/σ) - erf((t - t₂)/σ) )

Us = []

amp_noises = linspace(-0.005,0.005,101)
for amp_noise = amp_noises
    cqs = CompositeQSystem([q0, q1])
    add_hamiltonian!(cqs, q0)
    # add rotating frame Hamiltonian shifts
    add_hamiltonian!(cqs, -q0.frequency*number(q0), q0)
    q1_freq = hamiltonian(q1, 0.0)[2,2]
    add_hamiltonian!(cqs, -q1_freq*number(q1), q1)

    diff_freq = q0.frequency - q1_freq
    add_hamiltonian!(cqs, rotating_flip_flop(q0, q1, 0.006, diff_freq), [q0,q1])

    add_hamiltonian!(cqs, flux_drive(q1, t -> amp*(1+amp_noise)*erf_squared(t)*sin(2π*freq*t)), q1)

    push!(Us, unitary_propagator(cqs, float(tmax)));
end
z_corrs = res.minimizer
Us = [project_qubit_subspace(U) for U = Us]
Us = [Zrot(z_corrs[1]) ⊗ Zrot(z_corrs[2]) * U for U = Us];

In [ ]:
p = plot(1+amp_noises, [avgfidelity(liou(U), liou(CZ)) for U = Us])
xlabel!(p, "Pulse Amplitude")
ylabel!(p, "CZ Fidelity")

In [ ]:
σ = 0.0005
distribution = exp.(-0.5*(amp_noises/σ).^2)
scale!(distribution, 1/sum(distribution))
plot(amp_noises, distribution)

In [ ]:
fids = Float64[]
sigmas = linspace(1e-4, 3e-3, 101)
for σ = sigmas
    distribution = exp.(-0.5*(amp_noises/σ).^2)
    scale!(distribution, 1/sum(distribution))
    fid = avgfidelity(kraus2liou([√λ*U for (λ,U) = zip(distribution, Us)]), liou(CZ))
    push!(fids, fid)
end
p = plot(sigmas, fids)
xlabel!(p, "Relative Pulse Amplitude σ")
ylabel!(p, "CZ Average Fidelity")